In [4]:
import csv
from pprint import pprint

file_path = "./data/24.csv"

data_dict = {}

with open(file_path, "r") as f:
    csv_reader = csv.reader(f)
    keys = next(csv_reader)
    for row in csv_reader:
        data_dict[row[0]] = dict(zip(keys, row))
        
pprint(data_dict)

{'1': {'1-sigma Mean (s)': '4.67',
       '1-sigma STD (s)': '1.48',
       'AMT (s)': '4.4',
       'Puzzles': '1 1 4 6',
       'Rank': '1',
       'Solved rate': '99.20%'},
 '10': {'1-sigma Mean (s)': '5.18',
        '1-sigma STD (s)': '1.63',
        'AMT (s)': '4.86',
        'Puzzles': '1 1 1 12',
        'Rank': '10',
        'Solved rate': '99.20%'},
 '100': {'1-sigma Mean (s)': '5.97',
         '1-sigma STD (s)': '1.91',
         'AMT (s)': '5.65',
         'Puzzles': '5 5 7 7',
         'Rank': '100',
         'Solved rate': '97.80%'},
 '1000': {'1-sigma Mean (s)': '10.72',
          '1-sigma STD (s)': '4.8',
          'AMT (s)': '11.28',
          'Puzzles': '4 9 10 13',
          'Rank': '1000',
          'Solved rate': '82.40%'},
 '1001': {'1-sigma Mean (s)': '11.38',
          '1-sigma STD (s)': '5.1',
          'AMT (s)': '11.31',
          'Puzzles': '3 3 5 12',
          'Rank': '1001',
          'Solved rate': '86.70%'},
 '1002': {'1-sigma Mean (s)': '11.66',
        

In [5]:
puzzle = data_dict['1']['Puzzles']
pprint(puzzle)

'1 1 4 6'


In [1]:
import guidance
from guidance import models, gen, select

lm = models.Transformers("microsoft/phi-2")

/mnt/d/Projects/forge-notebooks/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.91s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [26]:
query = f"Use numbers and basic arithmetic operations (+ - * /) to obtain 24. {puzzle}"

puzzle_list = [int(char) for char in puzzle.split()]
history = None
total = None

objective = ""

rules = ""

strategies = ""

attempted = ""

game_of_24 = lm + f"""\
Objective: {objective} \n
Rules: {rules} \n
Strategies: {strategies} \n
History: {history} \n
Query: {query} \n
Total: {total}
"""

game_of_24

In [ ]:
# Solver loop

puzzle_state = {
                'total': 0,
                'left': puzzle_list,
                'solved': False,
                'max_attempts': 10,
                'history': []
            }

i = 0

if not puzzle_state['solved'] or i < puzzle_state['max_attempts']:
    while puzzle_list:
        if total == None:            
            first = game_of_24 + select(puzzle_list, name = 'selected')
            total = int(first['selected'])
            puzzle_state['total'] = total
            
            first_idx = puzzle_list.index(first['selected'])
            puzzle_list.pop(first_idx)
            puzzle_state['left'] = puzzle_list

        else:
            #  = game_of_24 + select(puzzle_list, name = 'selected')
    
            action = game_of_24 + f"""\
            selection number
            selection operation
            combine for total
            
            """
            
            total = int(action['total'])
            puzzle_state['total'] = total
            
            action_idx = puzzle_list.index(action['selected'])
            puzzle_list.pop(action_idx)
            puzzle_state['left'] = puzzle_list
                      
    if total == 24:
        puzzle_state['solved'] = True
    
    i += 1

In [ ]:
@guidance
def react_prompt_example(lm, question, max_rounds=10):
    lm += f'Question: {question}\n'
    i = 1
    while True:
        lm += f'Thought {i}: ' + gen(suffix='\n')
        lm += f'Act {i}: ' + select(['Search', 'Finish'], name='act') 
        lm += '[' + gen(name='arg', suffix=']') + '\n'
        if lm['act'] == 'Finish' or i == max_rounds:
            break
        else:
            lm += f'Observation {i}: ' + search(lm['arg']) + '\n'
        i += 1
    return lm